# HER2 One Scanner - Aperio FDA

- 5-Fold (80/20) split, No Holdout Set
- Truth = Categorical from Mean of 7 continuous scores 
- Epoch at automatic Stop when loss<.001 change 
- LeNet model, 10 layers, Dropout (0.7)

In [1]:
import numpy as np
import pandas as pd
import random
from keras.callbacks import EarlyStopping
from PIL import Image
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Lambda
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve, auc, classification_report
import csv
import cv2
import scipy
import os
%matplotlib inline
import matplotlib.pyplot as plt

Using TensorFlow backend.
/usr/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#For single scanner
BASE_PATH = '/home/diam/Desktop/HER2_data_augmented/Aperio_FDA/'
#BASE PATH for working from home:
#BASE_PATH = '/home/OSEL/Desktop/HER2_data_categorical/'
#epochs = 10
batch_size = 32
num_classes = 3
#epochs = 35

## Get Data - Practice

In [3]:
#This is the version from Ravi's code:

#FDA
#X_FDA = []
#idx_FDA = []
#for index, image_filename in list(enumerate(BASE_PATH)):
#	img_file = cv2.imread(BASE_PATH + '/' + image_filename)
#	if img_file is not None:
		#img_file = smisc.imresize(arr = img_file, size = (600,760,3))
#		img_file = smisc.imresize(arr = img_file, size = (120,160,3))		
#		img_arr = np.asarray(img_file)
#		X_FDA.append(img_arr)
#		idx_FDA.append(index)

#X_FDA = np.asarray(X_FDA)
#idx_FDA = np.asarray(idx_FDA)
#random.seed(rs)
#random_id = random.sample(idx_FDA, len(idx_FDA)/2)
#random_FDA = []
#for i in random_id:
#	random_FDA.append(X_FDA[i])

#random_FDA = np.asarray(random_FDA)

## Get Data - Real

In [4]:
def get_data(folder):
    X = []
    y = []
    filenames = []

    for hclass in os.listdir(folder):
        if not hclass.startswith('.'):
            if hclass in ["1"]:
                label = 1
            else: #label must be 1 or 2
                if hclass in ["2"]:
                    label = 2
                else:
                    label = 3
            for image_filename in os.listdir(folder + hclass):
                filename = folder + hclass + '/' + image_filename
                img_file = cv2.imread(folder + hclass + '/' + image_filename)
                
                if img_file is not None:
                    img_file = scipy.misc.imresize(arr=img_file, size=(120, 160, 3))
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(label)
                    filenames.append(filename)

    X = np.asarray(X)
    y = np.asarray(y)
    z = np.asarray(filenames)
    return X,y,z

In [5]:
X, y, z = get_data(BASE_PATH)


#print(X)
#print(y)
#print(z)
print(len(X))
print(len(y))
print(len(z))

#INTEGER ENCODE
#https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/
encoder = LabelEncoder()
y_cat = np_utils.to_categorical(encoder.fit_transform(y))
print(y_cat)





/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


305
305
305
[[ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.

### Old Code

In [6]:

#encoder = LabelEncoder()
#encoder.fit(y)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

#encoded_y_train = encoder.transform(y_train)
#encoded_y_test = encoder.transform(y_test)

#y_train = np_utils.to_categorical(encoded_y_train)
#y_test = np_utils.to_categorical(encoded_y_test)

In [7]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

## Fit Model with K-Fold X-Val

In [8]:
kf = KFold(n_splits = 305, random_state=5, shuffle=True)
print(kf.get_n_splits(y))
print(kf)


#for train_index, test_index in kf.split(y):
#    X_train, X_test = X[train_index], X[test_index]
#    print(train_index, test_index)




305
KFold(n_splits=305, random_state=5, shuffle=True)


In [9]:

oos_y = []
oos_pred = []
fold = 0


for train, test in kf.split(y_cat):
    fold+=1
    print("fold #{}".format(fold))
    
    random.shuffle(train)
    random.shuffle(test)
    
    X_train = X[train]
    y_train = y_cat[train]
    X_test = X[test]
    y_test = y_cat[test]
    
    
    model = Sequential()
    model.add(Lambda(lambda x: x * 1./255., input_shape=(120, 160, 3), output_shape=(120, 160, 3)))
    model.add(Conv2D(32, (3, 3), input_shape=(120, 160, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.7))
    model.add(Dense(3))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])
    
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=25, verbose=1, mode='auto')
   
    
    model.fit(
            X_train,
            y_train,
            validation_data=(X_test,y_test),
            callbacks=[monitor],
            shuffle=True,
            batch_size=batch_size,
            verbose=0,
            epochs=1000)
        
    pred = model.predict(X_test)
        
    oos_y.append(y_test)
    #pred = np.argmax(pred)
    predx = np.argmax(pred,axis=1)
    oos_pred.append(predx)
    
        #measure the fold's accuracy
    y_compare = np.argmax(y_test,axis=1) #for accuracy calculation
    score = metrics.accuracy_score(y_compare, predx)
    print("Fold Score (accuracy): {}".format(score))
    
    print(y_test)
    #print(test)
    print(pred)
    print(predx)
    #print(z[test])
    #print(oos_y)
    
    
    #outlist = []
    
    #outline = index[0] + '\t' + str(z[test]) + '\t' + str(int(pred[0])) + '\t' + str(int(pred[1])) + '\t' + str(int(pred[2])) + '\t' + str(int(y[test])) + '\t' + str(predx) + \n'
    #outlist.append(outline)
    #with open("results.txt", "w") as f:
        #f.writelines(outlist)
        #f.writelines(z[test])

fold #1
Epoch 00081: early stopping
Fold Score (accuracy): 1.0
[[ 0.  1.  0.]]
[[  5.17372209e-05   9.99897122e-01   5.10833306e-05]]
[1]
fold #2
Epoch 00027: early stopping
Fold Score (accuracy): 0.0
[[ 0.  1.  0.]]
[[ 0.00209497  0.41113618  0.58676881]]
[2]
fold #3
Epoch 00044: early stopping
Fold Score (accuracy): 0.0
[[ 0.  0.  1.]]
[[  6.90236388e-13   9.98480141e-01   1.51989690e-03]]
[1]
fold #4
Epoch 00076: early stopping
Fold Score (accuracy): 0.0
[[ 1.  0.  0.]]
[[ 0.12862791  0.86760151  0.00377062]]
[1]
fold #5
Epoch 00029: early stopping
Fold Score (accuracy): 1.0
[[ 1.  0.  0.]]
[[  1.00000000e+00   3.75530870e-21   2.54284024e-21]]
[0]
fold #6
Epoch 00035: early stopping
Fold Score (accuracy): 1.0
[[ 0.  0.  1.]]
[[  8.64286448e-21   3.91668655e-05   9.99960780e-01]]
[2]
fold #7
Epoch 00039: early stopping
Fold Score (accuracy): 1.0
[[ 0.  0.  1.]]
[[  1.69804781e-09   1.62188277e-01   8.37811708e-01]]
[2]
fold #8
Epoch 00030: early stopping
Fold Score (accuracy): 1.0
[

Epoch 00050: early stopping
Fold Score (accuracy): 1.0
[[ 0.  0.  1.]]
[[  1.37248094e-15   2.21357662e-02   9.77864206e-01]]
[2]
fold #63
Epoch 00029: early stopping
Fold Score (accuracy): 1.0
[[ 0.  0.  1.]]
[[  1.66618741e-09   1.10970922e-02   9.88902867e-01]]
[2]
fold #64
Epoch 00033: early stopping
Fold Score (accuracy): 1.0
[[ 0.  1.  0.]]
[[ 0.15390849  0.81993818  0.02615336]]
[1]
fold #65
Epoch 00105: early stopping
Fold Score (accuracy): 1.0
[[ 0.  1.  0.]]
[[  1.51564673e-05   9.99419332e-01   5.65535505e-04]]
[1]
fold #66
Epoch 00035: early stopping
Fold Score (accuracy): 1.0
[[ 0.  1.  0.]]
[[ 0.00576043  0.7147249   0.2795147 ]]
[1]
fold #67
Epoch 00031: early stopping
Fold Score (accuracy): 0.0
[[ 1.  0.  0.]]
[[ 0.3013908   0.65233558  0.04627356]]
[1]
fold #68
Epoch 00029: early stopping
Fold Score (accuracy): 1.0
[[ 1.  0.  0.]]
[[  1.00000000e+00   7.94764109e-24   7.06472214e-20]]
[0]
fold #69
Epoch 00072: early stopping
Fold Score (accuracy): 1.0
[[ 0.  1.  0.]]
[

Epoch 00046: early stopping
Fold Score (accuracy): 1.0
[[ 0.  0.  1.]]
[[  7.07171992e-08   2.90037505e-02   9.70996201e-01]]
[2]
fold #124
Epoch 00061: early stopping
Fold Score (accuracy): 1.0
[[ 0.  1.  0.]]
[[  1.39890808e-05   9.99596894e-01   3.89165420e-04]]
[1]
fold #125
Epoch 00080: early stopping
Fold Score (accuracy): 1.0
[[ 0.  1.  0.]]
[[  7.70834321e-03   9.92288768e-01   2.83593408e-06]]
[1]
fold #126
Epoch 00032: early stopping
Fold Score (accuracy): 1.0
[[ 0.  0.  1.]]
[[  2.02966916e-33   1.83569815e-09   1.00000000e+00]]
[2]
fold #127
Epoch 00028: early stopping
Fold Score (accuracy): 1.0
[[ 1.  0.  0.]]
[[  1.00000000e+00   1.17382048e-21   1.37802801e-27]]
[0]
fold #128
Epoch 00026: early stopping
Fold Score (accuracy): 1.0
[[ 0.  1.  0.]]
[[ 0.04826087  0.84653705  0.10520215]]
[1]
fold #129
Epoch 00079: early stopping
Fold Score (accuracy): 1.0
[[ 0.  1.  0.]]
[[  1.49403850e-03   9.98505712e-01   2.84101390e-07]]
[1]
fold #130
Epoch 00029: early stopping
Fold Sc

Epoch 00029: early stopping
Fold Score (accuracy): 1.0
[[ 0.  0.  1.]]
[[  3.30223023e-12   9.79626420e-05   9.99902010e-01]]
[2]
fold #184
Epoch 00070: early stopping
Fold Score (accuracy): 1.0
[[ 0.  1.  0.]]
[[  1.99371379e-14   9.66390431e-01   3.36095728e-02]]
[1]
fold #185
Epoch 00119: early stopping
Fold Score (accuracy): 1.0
[[ 0.  1.  0.]]
[[  9.34360269e-03   9.90355492e-01   3.00970074e-04]]
[1]
fold #186
Epoch 00028: early stopping
Fold Score (accuracy): 1.0
[[ 1.  0.  0.]]
[[  1.00000000e+00   1.21883719e-18   2.53369200e-27]]
[0]
fold #187
Epoch 00085: early stopping
Fold Score (accuracy): 0.0
[[ 0.  1.  0.]]
[[  1.53645361e-19   6.27342843e-06   9.99993682e-01]]
[2]
fold #188
Epoch 00065: early stopping
Fold Score (accuracy): 0.0
[[ 0.  0.  1.]]
[[  2.22921237e-10   9.09491956e-01   9.05080438e-02]]
[1]
fold #189
Epoch 00029: early stopping
Fold Score (accuracy): 1.0
[[ 1.  0.  0.]]
[[  1.00000000e+00   4.56279751e-24   1.75494167e-29]]
[0]
fold #190
Epoch 00032: early s

Epoch 00030: early stopping
Fold Score (accuracy): 1.0
[[ 1.  0.  0.]]
[[  1.00000000e+00   6.86969264e-24   7.93995088e-30]]
[0]
fold #244
Epoch 00035: early stopping
Fold Score (accuracy): 1.0
[[ 0.  0.  1.]]
[[  3.06171029e-19   7.99275426e-07   9.99999166e-01]]
[2]
fold #245
Epoch 00095: early stopping
Fold Score (accuracy): 0.0
[[ 0.  0.  1.]]
[[  2.90834423e-09   9.40093994e-01   5.99060617e-02]]
[1]
fold #246
Epoch 00028: early stopping
Fold Score (accuracy): 1.0
[[ 0.  0.  1.]]
[[  7.70288056e-16   5.83443523e-07   9.99999404e-01]]
[2]
fold #247
Epoch 00062: early stopping
Fold Score (accuracy): 0.0
[[ 0.  0.  1.]]
[[  1.10446090e-04   9.97525394e-01   2.36416166e-03]]
[1]
fold #248
Epoch 00039: early stopping
Fold Score (accuracy): 1.0
[[ 0.  0.  1.]]
[[  5.69690643e-15   3.25360634e-05   9.99967456e-01]]
[2]
fold #249
Epoch 00091: early stopping
Fold Score (accuracy): 1.0
[[ 0.  1.  0.]]
[[  4.34957947e-05   9.99916434e-01   4.00487297e-05]]
[1]
fold #250
Epoch 00029: early s

Epoch 00063: early stopping
Fold Score (accuracy): 1.0
[[ 0.  1.  0.]]
[[ 0.00925578  0.94050062  0.05024357]]
[1]
fold #304
Epoch 00054: early stopping
Fold Score (accuracy): 1.0
[[ 0.  0.  1.]]
[[  3.06544627e-23   8.12315477e-08   9.99999881e-01]]
[2]
fold #305
Epoch 00042: early stopping
Fold Score (accuracy): 1.0
[[ 0.  0.  1.]]
[[  1.14051691e-29   1.90674036e-04   9.99809325e-01]]
[2]
